In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the datasets into a list
df_list = [
    pd.read_csv('/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/2015-16-5.csv'),
    pd.read_csv('/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/2013-1.csv'),
    pd.read_csv('/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/2014-2.csv'),
    pd.read_csv('/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/2016-17-3.csv'),
    pd.read_csv('/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/2018-19-4.csv')
]

# Assign to individual DataFrames
df1, df2, df3, df4, df5 = df_list

# Print column names for each DataFrame
for i, df in enumerate(df_list, start=1):
    print(f"df{i} columns: {df.columns.unique()}\n")

df1 columns: Index(['ExtractDate', 'OrganisationURI', 'OrganisationLabel', 'ServiceTypeURI',
       'ServiceTypeLabel', 'Fleet', 'Vehicle', 'Fuel', 'Dist.Run', 'MPG',
       'Typ'],
      dtype='object')

df2 columns: Index(['ExtractDate', 'OrganisationURI', 'OrganisationLabel', 'ServiceTypeURI',
       'ServiceTypeLabel', 'Fleet', 'Vehicle', 'Fuel', 'Dist.Run', 'MPG',
       'Type'],
      dtype='object')

df3 columns: Index(['ExtractDate', 'OrganisationURI', 'OrganisationLabel', 'ServiceTypeURI',
       'ServiceTypeLabel', 'Fleet', 'Vehicle', 'Fuel', 'Dist.Run', 'MPG',
       'Type'],
      dtype='object')

df4 columns: Index(['Reference', 'Fleet', 'Vehicle', 'Fuel', 'Typ', 'Odo', 'Dist.Run',
       'MPG'],
      dtype='object')

df5 columns: Index(['Number', 'Registration', 'Details', 'Product', 'Odometer', 'Distance',
       'Unit', 'MPG'],
      dtype='object')



In [3]:
# Standardize Column Names Function
def standardize_columns(df):
    # Convert to lowercase
    df.columns = df.columns.str.lower()
    
    # Replace spaces and special characters
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('.', '_')
    
    return df
    
# Columns to drop
drop_cols = ["organisationuri", "organisationlabel", "servicetypeuri", "servicetypelabel", "number"]

# Standardize columns for each DataFrame and drop specified columns
for i, df in enumerate(df_list):
    # Standardize columns
    df = standardize_columns(df)
    
    # Drop specified columns if they exist
    for col in drop_cols:
        if col in df.columns:
            df = df.drop(columns=[col])
    
    # Reassign the modified DataFrame back to the list
    df_list[i] = df
    
    # Print the standardized column names
    print(f"Standardized columns for df{i}: {df.columns.unique()}\n")

# Unpack the DataFrames back to individual variables if needed
df1, df2, df3, df4, df5 = df_list


Standardized columns for df0: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'dist_run', 'mpg', 'typ'], dtype='object')

Standardized columns for df1: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'dist_run', 'mpg', 'type'], dtype='object')

Standardized columns for df2: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'dist_run', 'mpg', 'type'], dtype='object')

Standardized columns for df3: Index(['reference', 'fleet', 'vehicle', 'fuel', 'typ', 'odo', 'dist_run',
       'mpg'],
      dtype='object')

Standardized columns for df4: Index(['registration', 'details', 'product', 'odometer', 'distance', 'unit',
       'mpg'],
      dtype='object')



In [4]:
# Define a mapping for standardized column names
column_mapping = {
    'typ': 'type',  # Standardized name
    'odo': 'odometer',  # Standardized name
    'dist_run': 'distance_run',  # Standardized name
    'reference': 'registration',  # If applicable, map accordingly
    'distance': 'distance_run',  # If applicable, map accordingly
    'details' : 'vehicle',
}


for i, df in enumerate(df_list):
    # Rename columns based on mapping
    df.rename(columns=column_mapping, inplace=True)
    # Print the standardized column names
    print(f"Standardized columns for df{i}: {df.columns.unique()}\n")

Standardized columns for df0: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'distance_run', 'mpg',
       'type'],
      dtype='object')

Standardized columns for df1: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'distance_run', 'mpg',
       'type'],
      dtype='object')

Standardized columns for df2: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'distance_run', 'mpg',
       'type'],
      dtype='object')

Standardized columns for df3: Index(['registration', 'fleet', 'vehicle', 'fuel', 'type', 'odometer',
       'distance_run', 'mpg'],
      dtype='object')

Standardized columns for df4: Index(['registration', 'vehicle', 'product', 'odometer', 'distance_run',
       'unit', 'mpg'],
      dtype='object')



In [5]:
# List of DataFrames
df_list2 = [df1, df2, df3, df4]

# Function to clean distance values
def clean_distance(value):
    if isinstance(value, str):
        # Remove unwanted characters and extract numeric part and unit
        value = value.strip('*?')  # Remove trailing * or ?
        if value[-1] in ['K', 'M']:
            try:
                numeric_part = int(value[:-1])  # Extract numeric part
                unit_part = value[-1]  # Get the unit (K or M)
                return numeric_part, unit_part
            except ValueError:
                return np.nan, None  # Handle conversion errors for non-integer values
        elif value in ['ONLY', 'ERROR']:
            return np.nan, None  # Return NaN for unwanted values
        else:
            try:
                return int(value), None  # Return numeric part with no unit
            except ValueError:
                return np.nan, None  # Handle conversion errors for non-integer values
    return np.nan, None  # Default case for non-string values

# Iterate through the DataFrames and apply the function
for i, df in enumerate(df_list2):
    # Apply the function to the 'Dist.Run' column
    df[['dist_run', 'unit']] = df['distance_run'].apply(lambda x: pd.Series(clean_distance(x)))
    
    # Map units to full names
    df['unit'] = df['unit'].map({"K": "Kilometers", "M": "Miles"})
    
    # Print the modified DataFrame for verification
    print(f"\nModified DataFrame {i + 1}:")
    print(df[['distance_run', 'dist_run', 'unit']])
    df = df.drop(columns=['distance_run'], inplace=True)


Modified DataFrame 1:
     distance_run  dist_run        unit
0          13848K   13848.0  Kilometers
1           ERROR       NaN         NaN
2         11313M*   11313.0       Miles
3          2321M*    2321.0       Miles
4          3683M*    3683.0       Miles
...           ...       ...         ...
1247       5852M*    5852.0       Miles
1248        3856M    3856.0       Miles
1249      22116K*   22116.0  Kilometers
1250       4637M*    4637.0       Miles
1251        776M*     776.0       Miles

[1252 rows x 3 columns]

Modified DataFrame 2:
     distance_run  dist_run        unit
0           3673M    3673.0       Miles
1           7002M    7002.0       Miles
2          5285M*    5285.0       Miles
3            ONLY       NaN         NaN
4           9322M    9322.0       Miles
...           ...       ...         ...
1211       35693K   35693.0  Kilometers
1212        7255M    7255.0       Miles
1213        1496M    1496.0       Miles
1214       6196M?    6196.0       Miles
1215     

In [6]:
# Print the standardized column names
for i, df in enumerate(df_list):
    print(f"Standardized columns for df{i}: {df.columns.unique()}\n")

Standardized columns for df0: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit'],
      dtype='object')

Standardized columns for df1: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit'],
      dtype='object')

Standardized columns for df2: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit'],
      dtype='object')

Standardized columns for df3: Index(['registration', 'fleet', 'vehicle', 'fuel', 'type', 'odometer', 'mpg',
       'dist_run', 'unit'],
      dtype='object')

Standardized columns for df4: Index(['registration', 'vehicle', 'product', 'odometer', 'distance_run',
       'unit', 'mpg'],
      dtype='object')



In [7]:
# Identify all unique columns across all DataFrames
all_columns = set()
for df in df_list:
    all_columns.update(df.columns)

# Ensure each DataFrame has all columns, filling missing ones with NaN
for i, df in enumerate(df_list):
    df.reset_index(drop=True, inplace=True) # Reset index for each DataFrame to ensure uniquenes
    for col in all_columns:
        if col not in df.columns:
            df[col] = np.nan  # Add missing column with NaN values
    print(f"Standardized columns for df{i}: {df.columns.unique()}")

Standardized columns for df0: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit', 'distance_run', 'registration', 'odometer', 'product'],
      dtype='object')
Standardized columns for df1: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit', 'distance_run', 'registration', 'odometer', 'product'],
      dtype='object')
Standardized columns for df2: Index(['extractdate', 'fleet', 'vehicle', 'fuel', 'mpg', 'type', 'dist_run',
       'unit', 'distance_run', 'registration', 'odometer', 'product'],
      dtype='object')
Standardized columns for df3: Index(['registration', 'fleet', 'vehicle', 'fuel', 'type', 'odometer', 'mpg',
       'dist_run', 'unit', 'distance_run', 'extractdate', 'product'],
      dtype='object')
Standardized columns for df4: Index(['registration', 'vehicle', 'product', 'odometer', 'distance_run',
       'unit', 'mpg', 'fuel', 'type', 'extractdate', 'dist_run', 'fleet'],
      dtype='object')


In [8]:
# Concatenate DataFrames vertically
try:
    combined_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
    print("Combined DataFrame shape:", combined_df.shape)
except Exception as e:
    print(f"Error during concatenation: {e}")
    
combined_df

Combined DataFrame shape: (5833, 12)


,extractdate,fleet,vehicle,fuel,mpg,type,dist_run,unit,distance_run,registration,odometer,product
0,13/07/2016,84600,GULLY TANK,7756.47,5.04,D,13848.0,Kilometers,NaN,NaN,NaN,NaN
1,13/07/2016,AY61ESG,SMALL VAN,1079.63,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN
2,13/07/2016,AY61ESN,SMALL VAN,1209.89,42.5,D,11313.0,Miles,NaN,NaN,NaN,NaN
3,13/07/2016,BD64LFP,SMALL VAN,303.33,41,D,2321.0,Miles,NaN,NaN,NaN,NaN
4,13/07/2016,BD64LHW,SMALL VAN,475.13,39.32,D,3683.0,Miles,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5828,NaN,NaN,Renault Kerax Hookloader,NaN,5.19,NaN,NaN,Kilometres,23346.0,PO13AYT,184143.0,Diesel
5829,NaN,NaN,Renault Kerax Hookloader,NaN,5.05,NaN,NaN,Kilometres,17721.0,PO13AYV,167764.0,Diesel
5830,NaN,NaN,Mercedes Arocs Hookloader,NaN,5.39,NaN,NaN,Kilometres,29244.0,YE16YPP,80191.0,Diesel
5831,NaN,NaN,Mercedes Arocs Hookloader,NaN,5.23,NaN,NaN,Kilometres,24100.0,YE16YPR,73568.0,Diesel


In [9]:
# Conversion factor from kilometers to miles
conversion_factor_km_to_miles = 0.621371

# Convert distance_run from kilometers to miles
for index, row in combined_df.iterrows():
    if row['unit'] == 'Kilometers':
        # Convert dist_run to miles
        combined_df.at[index, 'dist_run'] = row['dist_run'] * conversion_factor_km_to_miles
    # Update unit to Miles
    combined_df.at[index, 'unit'] = 'Miles'

In [10]:
combined_df

,extractdate,fleet,vehicle,fuel,mpg,type,dist_run,unit,distance_run,registration,odometer,product
0,13/07/2016,84600,GULLY TANK,7756.47,5.04,D,8604.745608,Miles,NaN,NaN,NaN,NaN
1,13/07/2016,AY61ESG,SMALL VAN,1079.63,NaN,D,NaN,Miles,NaN,NaN,NaN,NaN
2,13/07/2016,AY61ESN,SMALL VAN,1209.89,42.5,D,11313.000000,Miles,NaN,NaN,NaN,NaN
3,13/07/2016,BD64LFP,SMALL VAN,303.33,41,D,2321.000000,Miles,NaN,NaN,NaN,NaN
4,13/07/2016,BD64LHW,SMALL VAN,475.13,39.32,D,3683.000000,Miles,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5828,NaN,NaN,Renault Kerax Hookloader,NaN,5.19,NaN,NaN,Miles,23346.0,PO13AYT,184143.0,Diesel
5829,NaN,NaN,Renault Kerax Hookloader,NaN,5.05,NaN,NaN,Miles,17721.0,PO13AYV,167764.0,Diesel
5830,NaN,NaN,Mercedes Arocs Hookloader,NaN,5.39,NaN,NaN,Miles,29244.0,YE16YPP,80191.0,Diesel
5831,NaN,NaN,Mercedes Arocs Hookloader,NaN,5.23,NaN,NaN,Miles,24100.0,YE16YPR,73568.0,Diesel


In [11]:
# Data Cleaning: Handle missing values if necessary
combined_df.dropna(subset=['fuel', 'mpg', 'dist_run'], inplace=True)  # Drop rows with critical missing values
combined_df['fleet'] = combined_df['fleet'].str.strip()  # removing whitespaces
combined_df['fleet'] = combined_df['fleet'].str.upper()  # Convert to uppercase for consistency

# Convert 'extractdate' to datetime format
combined_df['extractdate'] = pd.to_datetime(combined_df['extractdate'], format='%d/%m/%Y')

# Convert mpg to numeric
combined_df['mpg'] = pd.to_numeric(combined_df['mpg'], errors='coerce')  

combined_df.fillna(0, inplace=True)

/var/folders/tb/v8fb3k6j3756crzpl6v5__8h0000gn/T/ipykernel_6182/1696769317.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  combined_df.fillna(0, inplace=True)
/var/folders/tb/v8fb3k6j3756crzpl6v5__8h0000gn/T/ipykernel_6182/1696769317.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df.fillna(0, inplace=True)


In [12]:
combined_df

,extractdate,fleet,vehicle,fuel,mpg,type,dist_run,unit,distance_run,registration,odometer,product
0,2016-07-13 00:00:00,84600,GULLY TANK,7756.47,5.04,D,8604.745608,Miles,0.0,0,0,0
2,2016-07-13 00:00:00,AY61ESN,SMALL VAN,1209.89,42.50,D,11313.000000,Miles,0.0,0,0,0
3,2016-07-13 00:00:00,BD64LFP,SMALL VAN,303.33,41.00,D,2321.000000,Miles,0.0,0,0,0
4,2016-07-13 00:00:00,BD64LHW,SMALL VAN,475.13,39.32,D,3683.000000,Miles,0.0,0,0,0
5,2016-07-13 00:00:00,BF15MOV,MED VAN,753.72,28.06,D,4252.000000,Miles,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5088,0,YT64GFO,MED VAN,325.00,27.99,D,2001.000000,Miles,0.0,YT64GFO,9579,0
5089,0,4431,2 AXLE BIN,14272.07,2.08,D,6539.929775,Miles,0.0,YV15VVS,35756,0
5090,0,YX15RVE,CAR,275.09,42.61,D,2579.000000,Miles,0.0,YX15RVE,9079,0
5091,0,YX65XRN,SMALL VAN,140.92,39.45,D,893.000000,Miles,0.0,YX65XRN,11371,0


In [13]:
# Save the modified DataFrame to a CSV file
output_file_path = '/Users/shounak/Documents/Personal_Projects/fleet-optimization/data/modified_data.csv'
combined_df.to_csv(output_file_path, index=False)